In [2]:
import os,re,csv,string,syllables,math,nltk,nltk.data,io,markovify,sys,getopt,os.path,random,json,gensim,requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
from nltk.collocations import *
from nltk.tokenize import word_tokenize,RegexpTokenizer
from nltk.util import ngrams
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer,WordNetLemmatizer
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense,LSTM,Embedding

D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as

**<font color = blue size=5>音節數量計算函式、爬蟲方式找相似詞語函式</font>**

In [3]:
# 計算音節數量
def count_syllables(word):
    return len(re.findall('(?!e$)[aeiouy]+', word, re.I) + re.findall('^[^aeiouy]*e$', word, re.I))
def syllable_count(word):
    word = word.lower()
    punc = string.punctuation
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    if word in punc: # 如果詞語是標點符號，音節數量則為0
        count = 0
    return count

In [5]:
# 透過爬蟲方式找相似詞語
def crawl_synonyms(word):
    url = "https://www.thesaurus.com/browse/"
    url += word
    #print(url)
    r = requests.get(url, headers={'Connection':'close'})
    # r = requests.get(url, timeout=5)
    soup = bs(r.content, "html.parser")
    answer_list = soup.find("ul",'css-17d6qyx-WordGridLayoutBox et6tpn80')
    try:
        syns = answer_list.findChildren("a")
        #print(syns)
    except:
        syns = []
    lst = []
    for i in range(len(syns)):
        lst.append(syns[i].text)
    return lst

**<font color = blue size=5>詞語清單，隨機挑選</font>**

In [32]:
# 將詞語依據音節數分類，產生s1 s2 s3 s4 s5 s6
def random_word_syllablelist():
    nouns = [
        'people',
        'history',
        'way',
        'art',
        'world',
        'information',
        'map',
        'two',
        'family',
        'government',
        'health',
        'system',
        'computer',
        'meat',
        'year',
        'thanks',
        'music',
        'person',
        'reading',
        'method',
        'data',
        'food',
        'understanding',
        'theory',
        'law',
        'bird',
        'literature',
        'problem',
        'software',
        'control',
        'knowledge',
        'power',
        'ability',
        'economics',
        'love',
        'internet',
        'television',
        'science',
        'library',
        'nature',
        'fact',
        'product',
        'idea',
        'temperature',
        'investment',
        'area',
        'society',
        'activity',
        'story',
        'industry',
        'media',
        'thing',
        'oven',
        'community',
        'definition',
        'safety',
        'quality',
        'development',
        'language',
        'management',
        'player',
        'variety',
        'video',
        'week',
        'security',
        'country',
        'exam',
        'movie',
        'organization',
        'equipment',
        'physics',
        'analysis',
        'policy',
        'series',
        'thought',
        'basis',
        'boyfriend',
        'direction',
        'strategy',
        'technology',
        'army',
        'camera',
        'freedom',
        'paper',
        'environment',
        'child',
        'instance',
        'month',
        'truth',
        'marketing',
        'university',
        'writing',
        'article',
        'department',
        'difference',
        'goal',
        'news',
        'audience',
        'fishing',
        'growth',
        'income',
        'marriage',
        'user',
        'combination',
        'failure',
        'meaning',
        'medicine',
        'philosophy',
        'teacher',
        'communication',
        'night',
        'chemistry',
        'disease',
        'disk',
        'energy',
        'nation',
        'road',
        'role',
        'soup',
        'advertising',
        'location',
        'success',
        'addition',
        'apartment',
        'education',
        'math',
        'moment',
        'painting',
        'politics',
        'attention',
        'decision',
        'event',
        'property',
        'shopping',
        'student',
        'wood',
        'competition',
        'distribution',
        'entertainment',
        'office',
        'population',
        'president',
        'unit',
        'category',
        'cigarette',
        'context',
        'introduction',
        'opportunity',
        'performance',
        'driver',
        'flight',
        'length',
        'magazine',
        'newspaper',
        'relationship',
        'teaching',
        'cell',
        'dealer',
        'debate',
        'finding',
        'lake',
        'member',
        'message',
        'phone',
        'scene',
        'appearance',
        'association',
        'concept',
        'customer',
        'death',
        'discussion',
        'housing',
        'inflation',
        'insurance',
        'mood',
        'woman',
        'advice',
        'blood',
        'effort',
        'expression',
        'importance',
        'opinion',
        'payment',
        'reality',
        'responsibility',
        'situation',
        'skill',
        'statement',
        'wealth',
        'application',
        'city',
        'county',
        'depth',
        'estate',
        'foundation',
        'grandmother',
        'heart',
        'perspective',
        'photo',
        'recipe',
        'studio',
        'topic',
        'collection',
        'depression',
        'imagination',
        'passion',
        'percentage',
        'resource',
        'setting',
        'ad',
        'agency',
        'college',
        'connection',
        'criticism',
        'debt',
        'description',
        'memory',
        'patience',
        'secretary',
        'solution',
        'administration',
        'aspect',
        'attitude',
        'director',
        'personality',
        'psychology',
        'recommendation',
        'response',
        'selection',
        'storage',
        'version',
        'alcohol',
        'argument',
        'complaint',
        'contract',
        'emphasis',
        'highway',
        'loss',
        'membership',
        'possession',
        'preparation',
        'steak',
        'union',
        'agreement',
        'cancer',
        'currency',
        'employment',
        'engineering',
        'entry',
        'interaction',
        'limit',
        'mixture',
        'preference',
        'region',
        'republic',
        'seat',
        'tradition',
        'virus',
        'actor',
        'classroom',
        'delivery',
        'device',
        'difficulty',
        'drama',
        'election',
        'engine',
        'football',
        'guidance',
        'hotel',
        'match',
        'owner',
        'priority',
        'protection',
        'suggestion',
        'tension',
        'variation',
        'anxiety',
        'atmosphere',
        'awareness',
        'bread',
        'climate',
        'comparison',
        'confusion',
        'construction',
        'elevator',
        'emotion',
        'employee',
        'employer',
        'guest',
        'height',
        'leadership',
        'mall',
        'manager',
        'operation',
        'recording',
        'respect',
        'sample',
        'transportation',
        'boring',
        'charity',
        'cousin',
        'disaster',
        'editor',
        'efficiency',
        'excitement',
        'extent',
        'feedback',
        'guitar',
        'homework',
        'leader',
        'mom',
        'outcome',
        'permission',
        'presentation',
        'promotion',
        'reflection',
        'refrigerator',
        'resolution',
        'revenue',
        'session',
        'singer',
        'tennis',
        'basket',
        'bonus',
        'cabinet',
        'childhood',
        'church',
        'clothes',
        'coffee',
        'dinner',
        'drawing',
        'hair',
        'hearing',
        'initiative',
        'judgment',
        'lab',
        'measurement',
        'mode',
        'mud',
        'orange',
        'poetry',
        'police',
        'possibility',
        'procedure',
        'queen',
        'ratio',
        'relation',
        'restaurant',
        'satisfaction',
        'sector',
        'signature',
        'significance',
        'song',
        'tooth',
        'town',
        'vehicle',
        'volume',
        'wife',
        'accident',
        'airport',
        'appointment',
        'arrival',
        'assumption',
        'baseball',
        'chapter',
        'committee',
        'conversation',
        'database',
        'enthusiasm',
        'error',
        'explanation',
        'farmer',
        'gate',
        'girl',
        'hall',
        'historian',
        'hospital',
        'injury',
        'instruction',
        'maintenance',
        'manufacturer',
        'meal',
        'perception',
        'pie',
        'poem',
        'presence',
        'proposal',
        'reception',
        'replacement',
        'revolution',
        'river',
        'son',
        'speech',
        'tea',
        'village',
        'warning',
        'winner',
        'worker',
        'writer',
        'assistance',
        'breath',
        'buyer',
        'chest',
        'chocolate',
        'conclusion',
        'contribution',
        'cookie',
        'courage',
        'dad',
        'desk',
        'drawer',
        'establishment',
        'examination',
        'garbage',
        'grocery',
        'honey',
        'impression',
        'improvement',
        'independence',
        'insect',
        'inspection',
        'inspector',
        'king',
        'ladder',
        'menu',
        'penalty',
        'piano',
        'potato',
        'profession',
        'professor',
        'quantity',
        'reaction',
        'requirement',
        'salad',
        'sister',
        'supermarket',
        'tongue',
        'weakness',
        'wedding',
        'affair',
        'ambition',
        'analyst',
        'apple',
        'assignment',
        'assistant',
        'bathroom',
        'bedroom',
        'beer',
        'birthday',
        'celebration',
        'championship',
        'cheek',
        'client',
        'consequence',
        'departure',
        'diamond',
        'dirt',
        'ear',
        'fortune',
        'friendship',
        'funeral',
        'gene',
        'girlfriend',
        'hat',
        'indication',
        'intention',
        'lady',
        'midnight',
        'negotiation',
        'obligation',
        'passenger',
        'pizza',
        'platform',
        'poet',
        'pollution',
        'recognition',
        'reputation',
        'shirt',
        'sir',
        'speaker',
        'stranger',
        'surgery',
        'sympathy',
        'tale',
        'throat',
        'trainer',
        'uncle',
        'youth',
        'time',
        'work',
        'film',
        'water',
        'money',
        'example',
        'while',
        'business',
        'study',
        'game',
        'life',
        'form',
        'air',
        'day',
        'place',
        'number',
        'part',
        'field',
        'fish',
        'back',
        'process',
        'heat',
        'hand',
        'experience',
        'job',
        'book',
        'end',
        'point',
        'type',
        'home',
        'economy',
        'value',
        'body',
        'market',
        'guide',
        'interest',
        'state',
        'radio',
        'course',
        'company',
        'price',
        'size',
        'card',
        'list',
        'mind',
        'trade',
        'line',
        'care',
        'group',
        'risk',
        'word',
        'fat',
        'force',
        'key',
        'light',
        'training',
        'name',
        'school',
        'top',
        'amount',
        'level',
        'order',
        'practice',
        'research',
        'sense',
        'service',
        'piece',
        'web',
        'boss',
        'sport',
        'fun',
        'house',
        'page',
        'term',
        'test',
        'answer',
        'sound',
        'focus',
        'matter',
        'kind',
        'soil',
        'board',
        'oil',
        'picture',
        'access',
        'garden',
        'range',
        'rate',
        'reason',
        'future',
        'site',
        'demand',
        'exercise',
        'image',
        'case',
        'cause',
        'coast',
        'action',
        'age',
        'bad',
        'boat',
        'record',
        'result',
        'section',
        'building',
        'mouse',
        'cash',
        'class',
        'nothing',
        'period',
        'plan',
        'store',
        'tax',
        'side',
        'subject',
        'space',
        'rule',
        'stock',
        'weather',
        'chance',
        'figure',
        'man',
        'model',
        'source',
        'beginning',
        'earth',
        'program',
        'chicken',
        'design',
        'feature',
        'head',
        'material',
        'purpose',
        'question',
        'rock',
        'salt',
        'act',
        'birth',
        'car',
        'dog',
        'object',
        'scale',
        'sun',
        'note',
        'profit',
        'rent',
        'speed',
        'style',
        'war',
        'bank',
        'craft',
        'half',
        'inside',
        'outside',
        'standard',
        'bus',
        'exchange',
        'eye',
        'fire',
        'position',
        'pressure',
        'stress',
        'advantage',
        'benefit',
        'box',
        'frame',
        'issue',
        'step',
        'cycle',
        'face',
        'item',
        'metal',
        'paint',
        'review',
        'room',
        'screen',
        'structure',
        'view',
        'account',
        'ball',
        'discipline',
        'medium',
        'share',
        'balance',
        'bit',
        'black',
        'bottom',
        'choice',
        'gift',
        'impact',
        'machine',
        'shape',
        'tool',
        'wind',
        'address',
        'average',
        'career',
        'culture',
        'morning',
        'pot',
        'sign',
        'table',
        'task',
        'condition',
        'contact',
        'credit',
        'egg',
        'hope',
        'ice',
        'network',
        'north',
        'square',
        'attempt',
        'date',
        'effect',
        'link',
        'post',
        'star',
        'voice',
        'capital',
        'challenge',
        'friend',
        'self',
        'shot',
        'brush',
        'couple',
        'exit',
        'front',
        'function',
        'lack',
        'living',
        'plant',
        'plastic',
        'spot',
        'summer',
        'taste',
        'theme',
        'track',
        'wing',
        'brain',
        'button',
        'click',
        'desire',
        'foot',
        'gas',
        'influence',
        'notice',
        'rain',
        'wall',
        'base',
        'damage',
        'distance',
        'feeling',
        'pair',
        'savings',
        'staff',
        'sugar',
        'target',
        'text',
        'animal',
        'author',
        'budget',
        'discount',
        'file',
        'ground',
        'lesson',
        'minute',
        'officer',
        'phase',
        'reference',
        'register',
        'sky',
        'stage',
        'stick',
        'title',
        'trouble',
        'bowl',
        'bridge',
        'campaign',
        'character',
        'club',
        'edge',
        'evidence',
        'fan',
        'letter',
        'lock',
        'maximum',
        'novel',
        'option',
        'pack',
        'park',
        'plenty',
        'quarter',
        'skin',
        'sort',
        'weight',
        'baby',
        'background',
        'carry',
        'dish',
        'factor',
        'fruit',
        'glass',
        'joint',
        'master',
        'muscle',
        'red',
        'strength',
        'traffic',
        'trip',
        'vegetable',
        'appeal',
        'chart',
        'gear',
        'ideal',
        'kitchen',
        'land',
        'log',
        'mother',
        'net',
        'party',
        'principle',
        'relative',
        'sale',
        'season',
        'signal',
        'spirit',
        'street',
        'tree',
        'wave',
        'belt',
        'bench',
        'commission',
        'copy',
        'drop',
        'minimum',
        'path',
        'progress',
        'project',
        'sea',
        'south',
        'status',
        'stuff',
        'ticket',
        'tour',
        'angle',
        'blue',
        'breakfast',
        'confidence',
        'daughter',
        'degree',
        'doctor',
        'dot',
        'dream',
        'duty',
        'essay',
        'father',
        'fee',
        'finance',
        'hour',
        'juice',
        'luck',
        'milk',
        'mouth',
        'peace',
        'pipe',
        'stable',
        'storm',
        'substance',
        'team',
        'trick',
        'afternoon',
        'bat',
        'beach',
        'blank',
        'catch',
        'chain',
        'consideration',
        'cream',
        'crew',
        'detail',
        'gold',
        'interview',
        'kid',
        'mark',
        'mission',
        'pain',
        'pleasure',
        'score',
        'screw',
        'sex',
        'shop',
        'shower',
        'suit',
        'tone',
        'window',
        'agent',
        'band',
        'bath',
        'block',
        'bone',
        'calendar',
        'candidate',
        'cap',
        'coat',
        'contest',
        'corner',
        'court',
        'cup',
        'district',
        'door',
        'east',
        'finger',
        'garage',
        'guarantee',
        'hole',
        'hook',
        'implement',
        'layer',
        'lecture',
        'lie',
        'manner',
        'meeting',
        'nose',
        'parking',
        'partner',
        'profile',
        'rice',
        'routine',
        'schedule',
        'swimming',
        'telephone',
        'tip',
        'winter',
        'airline',
        'bag',
        'battle',
        'bed',
        'bill',
        'bother',
        'cake',
        'code',
        'curve',
        'designer',
        'dimension',
        'dress',
        'ease',
        'emergency',
        'evening',
        'extension',
        'farm',
        'fight',
        'gap',
        'grade',
        'holiday',
        'horror',
        'horse',
        'host',
        'husband',
        'loan',
        'mistake',
        'mountain',
        'nail',
        'noise',
        'occasion',
        'package',
        'patient',
        'pause',
        'phrase',
        'proof',
        'race',
        'relief',
        'sand',
        'sentence',
        'shoulder',
        'smoke',
        'stomach',
        'string',
        'tourist',
        'towel',
        'vacation',
        'west',
        'wheel',
        'wine',
        'arm',
        'aside',
        'associate',
        'bet',
        'blow',
        'border',
        'branch',
        'breast',
        'brother',
        'buddy',
        'bunch',
        'chip',
        'coach',
        'cross',
        'document',
        'draft',
        'dust',
        'expert',
        'floor',
        'god',
        'golf',
        'habit',
        'iron',
        'judge',
        'knife',
        'landscape',
        'league',
        'mail',
        'mess',
        'native',
        'opening',
        'parent',
        'pattern',
        'pin',
        'pool',
        'pound',
        'request',
        'salary',
        'shame',
        'shelter',
        'shoe',
        'silver',
        'tackle',
        'tank',
        'trust',
        'assist',
        'bake',
        'bar',
        'bell',
        'bike',
        'blame',
        'boy',
        'brick',
        'chair',
        'closet',
        'clue',
        'collar',
        'comment',
        'conference',
        'devil',
        'diet',
        'fear',
        'fuel',
        'glove',
        'jacket',
        'lunch',
        'monitor',
        'mortgage',
        'nurse',
        'pace',
        'panic',
        'peak',
        'plane',
        'reward',
        'row',
        'sandwich',
        'shock',
        'spite',
        'spray',
        'surprise',
        'till',
        'transition',
        'weekend',
        'welcome',
        'yard',
        'alarm',
        'bend',
        'bicycle',
        'bite',
        'blind',
        'bottle',
        'cable',
        'candle',
        'clerk',
        'cloud',
        'concert',
        'counter',
        'flower',
        'grandfather',
        'harm',
        'knee',
        'lawyer',
        'leather',
        'load',
        'mirror',
        'neck',
        'pension',
        'plate',
        'purple',
        'ruin',
        'ship',
        'skirt',
        'slice',
        'snow',
        'specialist',
        'stroke',
        'switch',
        'trash',
        'tune',
        'zone',
        'anger',
        'award',
        'bid',
        'bitter',
        'boot',
        'bug',
        'camp',
        'candy',
        'carpet',
        'cat',
        'champion',
        'channel',
        'clock',
        'comfort',
        'cow',
        'crack',
        'engineer',
        'entrance',
        'fault',
        'grass',
        'guy',
        'hell',
        'highlight',
        'incident',
        'island',
        'joke',
        'jury',
        'leg',
        'lip',
        'mate',
        'motor',
        'nerve',
        'passage',
        'pen',
        'pride',
        'priest',
        'prize',
        'promise',
        'resident',
        'resort',
        'ring',
        'roof',
        'rope',
        'sail',
        'scheme',
        'script',
        'sock',
        'station',
        'toe',
        'tower',
        'truck',
        'witness',
        'a',
        'you',
        'it',
        'can',
        'will',
        'if',
        'one',
        'many',
        'most',
        'other',
        'use',
        'make',
        'good',
        'look',
        'help',
        'go',
        'great',
        'being',
        'few',
        'might',
        'still',
        'public',
        'read',
        'keep',
        'start',
        'give',
        'human',
        'local',
        'general',
        'she',
        'specific',
        'long',
        'play',
        'feel',
        'high',
        'tonight',
        'put',
        'common',
        'set',
        'change',
        'simple',
        'past',
        'big',
        'possible',
        'particular',
        'today',
        'major',
        'personal',
        'current',
        'national',
        'cut',
        'natural',
        'physical',
        'show',
        'try',
        'check',
        'second',
        'call',
        'move',
        'pay',
        'let',
        'increase',
        'single',
        'individual',
        'turn',
        'ask',
        'buy',
        'guard',
        'hold',
        'main',
        'offer',
        'potential',
        'professional',
        'international',
        'travel',
        'cook',
        'alternative',
        'following',
        'special',
        'working',
        'whole',
        'dance',
        'excuse',
        'cold',
        'commercial',
        'low',
        'purchase',
        'deal',
        'primary',
        'worth',
        'fall',
        'necessary',
        'positive',
        'produce',
        'search',
        'present',
        'spend',
        'talk',
        'creative',
        'tell',
        'cost',
        'drive',
        'green',
        'support',
        'glad',
        'remove',
        'return',
        'run',
        'complex',
        'due',
        'effective',
        'middle',
        'regular',
        'reserve',
        'independent',
        'leave',
        'original',
        'reach',
        'rest',
        'serve',
        'watch',
        'beautiful',
        'charge',
        'active',
        'break',
        'negative',
        'safe',
        'stay',
        'visit',
        'visual',
        'affect',
        'cover',
        'report',
        'rise',
        'walk',
        'white',
        'beyond',
        'junior',
        'pick',
        'unique',
        'anything',
        'classic',
        'final',
        'lift',
        'mix',
        'private',
        'stop',
        'teach',
        'western',
        'concern',
        'familiar',
        'fly',
        'official',
        'broad',
        'comfortable',
        'gain',
        'maybe',
        'rich',
        'save',
        'stand',
        'young',
        'heavy',
        'hello',
        'lead',
        'listen',
        'valuable',
        'worry',
        'handle',
        'leading',
        'meet',
        'release',
        'sell',
        'finish',
        'normal',
        'press',
        'ride',
        'secret',
        'spread',
        'spring',
        'tough',
        'wait',
        'brown',
        'deep',
        'display',
        'flow',
        'hit',
        'objective',
        'shoot',
        'touch',
        'cancel',
        'chemical',
        'cry',
        'dump',
        'extreme',
        'push',
        'conflict',
        'eat',
        'fill',
        'formal',
        'jump',
        'kick',
        'opposite',
        'pass',
        'pitch',
        'remote',
        'total',
        'treat',
        'vast',
        'abuse',
        'beat',
        'burn',
        'deposit',
        'print',
        'raise',
        'sleep',
        'somewhere',
        'advance',
        'anywhere',
        'consist',
        'dark',
        'double',
        'draw',
        'equal',
        'fix',
        'hire',
        'internal',
        'join',
        'kill',
        'sensitive',
        'tap',
        'win',
        'attack',
        'claim',
        'constant',
        'drag',
        'drink',
        'guess',
        'minor',
        'pull',
        'raw',
        'soft',
        'solid',
        'wear',
        'weird',
        'wonder',
        'annual',
        'count',
        'dead',
        'doubt',
        'feed',
        'forever',
        'impress',
        'nobody',
        'repeat',
        'round',
        'sing',
        'slide',
        'strip',
        'whereas',
        'wish',
        'combine',
        'command',
        'dig',
        'divide',
        'equivalent',
        'hang',
        'hunt',
        'initial',
        'march',
        'mention',
        'spiritual',
        'survey',
        'tie',
        'adult',
        'brief',
        'crazy',
        'escape',
        'gather',
        'hate',
        'prior',
        'repair',
        'rough',
        'sad',
        'scratch',
        'sick',
        'strike',
        'employ',
        'external',
        'hurt',
        'illegal',
        'laugh',
        'lay',
        'mobile',
        'nasty',
        'ordinary',
        'respond',
        'royal',
        'senior',
        'split',
        'strain',
        'struggle',
        'swim',
        'train',
        'upper',
        'wash',
        'yellow',
        'convert',
        'crash',
        'dependent',
        'fold',
        'funny',
        'grab',
        'hide',
        'miss',
        'permit',
        'quote',
        'recover',
        'resolve',
        'roll',
        'sink',
        'slip',
        'spare',
        'suspect',
        'sweet',
        'swing',
        'twist',
        'upstairs',
        'usual',
        'abroad',
        'brave',
        'calm',
        'concentrate',
        'estimate',
        'grand',
        'male',
        'mine',
        'prompt',
        'quiet',
        'refuse',
        'regret',
        'reveal',
        'rush',
        'shake',
        'shift',
        'shine',
        'steal',
        'suck',
        'surround',
        'anybody',
        'bear',
        'brilliant',
        'dare',
        'dear',
        'delay',
        'drunk',
        'female',
        'hurry',
        'inevitable',
        'invite',
        'kiss',
        'neat',
        'pop',
        'punch',
        'quit',
        'reply',
        'representative',
        'resist',
        'rip',
        'rub',
        'silly',
        'smile',
        'spell',
        'stretch',
        'stupid',
        'tear',
        'temporary',
        'tomorrow',
        'wake',
        'wrap',
        'yesterday']
    s1 = []
    s2 = []
    s3 = []
    s4 = []
    s5 = []
    s6 = []
    s1_s6 = []
    #print(" ".join([nouns[random.randrange(0, len(nouns))] for i in range(4)]))
    for a in range(len(nouns)):
        count = syllable_count(nouns[a])
        if count == 1:
            s1.append(nouns[a])
        if count == 2:
            s2.append(nouns[a])
        if count == 3:
            s3.append(nouns[a])
        if count == 4:
            s4.append(nouns[a])
        if count == 5:
            s5.append(nouns[a])
        if count == 6:
            s6.append(nouns[a])
    s1_s6.append(s1)
    s1_s6.append(s2)
    s1_s6.append(s3)
    s1_s6.append(s4)
    s1_s6.append(s5)
    s1_s6.append(s6)
    print(len(nouns),'(total) /',len(s1_s6),'(type) /',len(s1),'(s1) /',len(s2),'(s2) /',len(s3),'(s3) /',len(s4),'(s4) /',len(s5),'(s5) /',len(s6),'(s6)')
    print('s1 =',s1)
    print('s2 =',s2)
    print('s3 =',s3)
    print('s4 =',s4)
    print('s5 =',s5)
    print('s6 =',s6)
    return s1_s6
s1_s6 = random_word_syllablelist()
print(s1_s6)

1524 (total) / 6 (type) / 709 (s1) / 493 (s2) / 227 (s3) / 79 (s4) / 15 (s5) / 1 (s6)
s1 = ['people', 'way', 'art', 'world', 'map', 'two', 'health', 'meat', 'year', 'thanks', 'food', 'law', 'bird', 'love', 'science', 'fact', 'thing', 'player', 'week', 'movie', 'thought', 'child', 'month', 'truth', 'goal', 'news', 'growth', 'night', 'disk', 'road', 'role', 'soup', 'math', 'wood', 'flight', 'length', 'cell', 'lake', 'phone', 'scene', 'death', 'mood', 'blood', 'skill', 'wealth', 'depth', 'heart', 'ad', 'debt', 'loss', 'steak', 'seat', 'match', 'bread', 'employee', 'guest', 'height', 'mall', 'sample', 'mom', 'church', 'coffee', 'hair', 'lab', 'mode', 'mud', 'queen', 'song', 'tooth', 'town', 'wife', 'gate', 'girl', 'hall', 'meal', 'pie', 'poem', 'son', 'speech', 'tea', 'breath', 'buyer', 'chest', 'cookie', 'dad', 'desk', 'king', 'tongue', 'apple', 'beer', 'cheek', 'client', 'dirt', 'ear', 'gene', 'hat', 'poet', 'shirt', 'sir', 'tale', 'throat', 'uncle', 'youth', 'time', 'work', 'film', 'whi

[['people', 'way', 'art', 'world', 'map', 'two', 'health', 'meat', 'year', 'thanks', 'food', 'law', 'bird', 'love', 'science', 'fact', 'thing', 'player', 'week', 'movie', 'thought', 'child', 'month', 'truth', 'goal', 'news', 'growth', 'night', 'disk', 'road', 'role', 'soup', 'math', 'wood', 'flight', 'length', 'cell', 'lake', 'phone', 'scene', 'death', 'mood', 'blood', 'skill', 'wealth', 'depth', 'heart', 'ad', 'debt', 'loss', 'steak', 'seat', 'match', 'bread', 'employee', 'guest', 'height', 'mall', 'sample', 'mom', 'church', 'coffee', 'hair', 'lab', 'mode', 'mud', 'queen', 'song', 'tooth', 'town', 'wife', 'gate', 'girl', 'hall', 'meal', 'pie', 'poem', 'son', 'speech', 'tea', 'breath', 'buyer', 'chest', 'cookie', 'dad', 'desk', 'king', 'tongue', 'apple', 'beer', 'cheek', 'client', 'dirt', 'ear', 'gene', 'hat', 'poet', 'shirt', 'sir', 'tale', 'throat', 'uncle', 'youth', 'time', 'work', 'film', 'while', 'game', 'life', 'form', 'air', 'day', 'place', 'part', 'field', 'fish', 'back', 'heat

**<font color = blue size=5>分析生成歌詞和原始歌詞(句子詞語分割、音節數分析)</font>**

https://www.geeksforgeeks.org/nlp-how-tokenizing-text-sentence-words-works/<br>
https://stackoverflow.com/questions/15547409/how-to-get-rid-of-punctuation-using-nltk-tokenizer

In [33]:
# 分析生成歌詞
def analyze_gen(gen_text):
    # 切割成單個單個詞語
    tokenizer = RegexpTokenizer("[\w']+") 
    lst_gen_tokenize = tokenizer.tokenize(gen_text)
    print(len(lst_gen_tokenize))
    print(lst_gen_tokenize)
    # 計算音節數量
    lst_gen_syllables = []
    for i in range(len(lst_gen_tokenize)):
        count_gen = syllable_count(lst_gen_tokenize[i])
        lst_gen_syllables.append(count_gen)
    print('lst_gen_syllables:',len(lst_gen_syllables))
    print(lst_gen_syllables)
    return lst_gen_syllables

gen_text = "Where the old men pray\
            His children lived next door\
            They used to live in the woods\
            But this man's religious bone\
            He didn't have a prayer\
            No, he didn't have a prayer\
            He just said I've got to find my own way home\
            And I've got to find my own way home\
            If I can do the things he says\
            That I just can't understand\
            Then I'll just settle down and start living for today\
            Lord, how I'd love to settle down\
            And start living for today\
            Lord, how I'd love to settle down\
            But I don't have a prayer\
            I don't have a prayer\
            No, I don't have a prayer\
            Sometimes, I think of what a wife must be\
            And how I hate to do things\
            That he doesn't have to do\
            But then somehow, I think of what hope is\
            And do I have to start\
            Oh, oh, I'm ready to begin\
            Oh, how I'd love to settle down\
            I'm ready, ready, ready\
            If I can do the things he says\
            That I just can't understand\
            Then I'll just settle down and start living for today\
            Lord, how I'd love to settle down\
            And I don't have a prayer\
            No, I don't have a prayer\
            Always, always start the day\
            Lord, how I'd love to settle down\
            I'm ready, ready, ready\
            If I can do the things he says\
            That I just can't understand\
            Then I'll just settle down and start living for today\
            Lord, how I'd love to settle down\
            But I don't have a prayer No, I don't have a prayer\
            I can't do the things he says\
            That I just can't understand\
            Then I'll just settle down and start living for today"
lst_gen_syllables = analyze_gen(gen_text)

288
['Where', 'the', 'old', 'men', 'pray', 'His', 'children', 'lived', 'next', 'door', 'They', 'used', 'to', 'live', 'in', 'the', 'woods', 'But', 'this', "man's", 'religious', 'bone', 'He', "didn't", 'have', 'a', 'prayer', 'No', 'he', "didn't", 'have', 'a', 'prayer', 'He', 'just', 'said', "I've", 'got', 'to', 'find', 'my', 'own', 'way', 'home', 'And', "I've", 'got', 'to', 'find', 'my', 'own', 'way', 'home', 'If', 'I', 'can', 'do', 'the', 'things', 'he', 'says', 'That', 'I', 'just', "can't", 'understand', 'Then', "I'll", 'just', 'settle', 'down', 'and', 'start', 'living', 'for', 'today', 'Lord', 'how', "I'd", 'love', 'to', 'settle', 'down', 'And', 'start', 'living', 'for', 'today', 'Lord', 'how', "I'd", 'love', 'to', 'settle', 'down', 'But', 'I', "don't", 'have', 'a', 'prayer', 'I', "don't", 'have', 'a', 'prayer', 'No', 'I', "don't", 'have', 'a', 'prayer', 'Sometimes', 'I', 'think', 'of', 'what', 'a', 'wife', 'must', 'be', 'And', 'how', 'I', 'hate', 'to', 'do', 'things', 'That', 'he', "

In [34]:
# 分析原始歌詞
def analyze_ori(ori_text):
    # 切割成單個單個詞語
    tokenizer = RegexpTokenizer("[\w']+") 
    lst_ori_tokenize = tokenizer.tokenize(ori_text)
    print(len(lst_ori_tokenize))
    print(lst_ori_tokenize)
    # 計算音節數量
    lst_ori_syllables = []
    for i in range(len(lst_ori_tokenize)):
        count_ori = syllable_count(lst_ori_tokenize[i])
        lst_ori_syllables.append(count_ori)
    print('lst_ori_syllables:',len(lst_ori_syllables))
    print(lst_ori_syllables)
    return lst_ori_syllables

ori_text = "I'll let you show me his moves\
            Let you do what he taught you\
            Let you reminisce how you used, to do\
            Girl I?m open to anything that'll get you into that zone\
            And understand that we?re all alone\
            So you can slowly take off your clothes\
            Baby girl you know what's in store\
            And baby I will stay up all night\
            I've been goin' hard since last night\
            And I'mma go harder tonight\
            Wish you could see you through my eyes\
            Ooh I'm telling you this ain't the same\
            And I know he's still in your brain\
            I'm 'bout to burn that shit into flames\
            Once I'm in you\
            Baby\
            Forget what you know\
            Make yourself at home\
            'Cuz baby when I'm finished with ya you won't wanna go Outside\
            And I'mma work you like a pro, baby\
            And, you gon' take it like one\
            Yeah you gon' take it like one\
            And I'mma give it like you asked for it\
            Why, cuz you been talkin' 'bout it\
            I know you been talkin' 'bout it\
            Ooh baby when I'm done with you\
            You ain't saying nothing\
            Yeah you ain't saying nothing\
            Ooh baby when I'm done with you\
            You ain't saying nothing\
            Yeah you ain't saying nothing\
            You gon' make me show off\
            All the pain that you feel you can tell that we ain't making no love\
            But I'll pretend\
            Oh girl, I'll pretend\
            If you pretend then girl I'll pretend\
            Let's make it seem like we're all we need in the end\
            Forget what you know\
            Make yourself at home\
            'Cuz baby when I'm finished with ya you won't wanna go Outside"
lst_ori_syllables = analyze_ori(ori_text)

277
["I'll", 'let', 'you', 'show', 'me', 'his', 'moves', 'Let', 'you', 'do', 'what', 'he', 'taught', 'you', 'Let', 'you', 'reminisce', 'how', 'you', 'used', 'to', 'do', 'Girl', 'I', 'm', 'open', 'to', 'anything', "that'll", 'get', 'you', 'into', 'that', 'zone', 'And', 'understand', 'that', 'we', 're', 'all', 'alone', 'So', 'you', 'can', 'slowly', 'take', 'off', 'your', 'clothes', 'Baby', 'girl', 'you', 'know', "what's", 'in', 'store', 'And', 'baby', 'I', 'will', 'stay', 'up', 'all', 'night', "I've", 'been', "goin'", 'hard', 'since', 'last', 'night', 'And', "I'mma", 'go', 'harder', 'tonight', 'Wish', 'you', 'could', 'see', 'you', 'through', 'my', 'eyes', 'Ooh', "I'm", 'telling', 'you', 'this', "ain't", 'the', 'same', 'And', 'I', 'know', "he's", 'still', 'in', 'your', 'brain', "I'm", "'bout", 'to', 'burn', 'that', 'shit', 'into', 'flames', 'Once', "I'm", 'in', 'you', 'Baby', 'Forget', 'what', 'you', 'know', 'Make', 'yourself', 'at', 'home', "'Cuz", 'baby', 'when', "I'm", 'finished', 'wit

In [37]:
# 比對兩個音節個數陣列(gen長度>ori長度)
def compare_gen_ori(lst_gen_syllables,lst_ori_syllables):
    len_g = len(lst_gen_syllables)
    len_o = len(lst_ori_syllables)
    if len_g > len_o:
        lst_gen_syllables2 = lst_gen_syllables[:len_g-(len_g-len_o)]
    # print('lst_gen_syllables2:',len(lst_gen_syllables2))
    # print(lst_gen_syllables2)
    correct = 0
    wrong = 0
    lst_correct = []
    lst_wrong = []
    tmp_w = []
    tmp_wo = []
    tmp_s = []
    tmp_so = []
    for j in range(len(lst_gen_syllables2)):
        if lst_gen_syllables[j] == lst_ori_syllables[j]:
            tmp_w.append(lst_gen_tokenize[j])
            tmp_wo.append('')
            tmp_s.append(lst_gen_syllables[j])
            tmp_so.append('')
            correct += 1
            lst_correct.append(j)
            lst_wrong.append('')
        else:
            tmp_w.append('')
            tmp_wo.append(lst_gen_tokenize[j]) # 生成的詞語對不上原始歌詞的音節
            tmp_s.append('')
            tmp_so.append(lst_ori_syllables[j]) # 正確的音節數
            wrong += 1
            lst_correct.append('')
            lst_wrong.append(j)
    print('correct:',correct,'/ worng:',wrong)
    print('lst_wrong:',lst_wrong)
    print('tmp_w:',len(tmp_w)) # 有對齊的生成詞語
    print(tmp_w)
    print('tmp_wo:',len(tmp_wo)) # 沒對齊的生成詞語
    print(tmp_wo)
    print('tmp_s:',len(tmp_s)) # 有對齊的生成詞語音節數量
    print(tmp_s)
    print('tmp_so:',len(tmp_so)) # 沒對齊的(取原始歌詞詞語應有的音節數量)
    print(tmp_so)
    return tmp_w, tmp_wo, tmp_s, tmp_so

tmp_w, tmp_wo, tmp_s, tmp_so = compare_gen_ori(lst_gen_syllables,lst_ori_syllables)

correct: 213 / worng: 64
lst_wrong: ['', '', '', '', '', '', '', 7, '', '', '', 11, '', '', '', '', 16, '', '', 19, 20, '', '', '', '', 25, '', 27, '', '', '', 31, '', '', '', 35, '', '', '', '', 40, '', '', '', 44, '', '', '', 48, 49, '', '', '', '', '', '', '', 57, '', '', '', '', '', '', '', 65, '', '', '', '', '', '', 72, 73, 74, '', '', '', '', '', '', '', '', '', '', 85, 86, 87, '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 106, 107, '', '', '', '', 112, 113, '', '', '', '', 118, '', '', '', 122, '', '', 125, '', '', '', '', 130, '', 132, '', 134, '', 136, '', '', '', 140, '', '', '', '', '', '', '', '', '', '', '', 152, '', 154, '', 156, '', '', '', '', 161, '', 163, 164, 165, '', '', 168, '', '', '', '', '', '', 175, '', '', 178, 179, '', '', '', '', '', '', 186, '', '', '', '', '', '', 193, '', 195, '', '', '', '', '', '', '', '', 204, '', '', '', 208, '', '', '', '', '', '', '', '', '', '', '', '', 221, 222, 223, '', '', '', '', 228, '', '', '', '', 

**<font color = blue size=5>比對生成和原始歌詞的主程式</font>**

In [40]:
# 方法一(m1)：爬蟲抓取同義詞(使用thesaurus網站)
# 方法二(m2)：wordnet抓取同義詞(使用NLTK)
def main_comparsion(tmp_wo):
    lst_rewrite = []

    for num in range(len(tmp_wo)): # len(tmp_wo)
        word = tmp_wo[num] # 生成詞語，用於找相似詞
        s = tmp_so[num] # 原始歌詞正確的音節數量

        # 詞語
        if word != '':
            print('original word:',word)
            print('correct syllable count:',s)

            crawl = crawl_synonyms(word) # 方法一抓取list結果
            print('crawl:',crawl) # 方法一抓取結果(未篩選音節數)
            if len(crawl) == 0: # 方法一沒有爬蟲結果
                s1_s6 = random_word_syllablelist()
                ran = random.choice(s1_s6[s-1]) # 根據音節數，到特定的音節陣列中隨機挑選
                crawl.append(ran)
            match_word_1 = []
            for n in range(len(crawl)):
                if syllable_count(crawl[n]) == s: # 方法一列出符合音節數的同義詞
                    match_word_1.append(crawl[n])
            try:
                print('m1 match list:',match_word_1) # 方法一篩選後的
                lst_rewrite.append(match_word_1[0])
            except:
                pass

            # 方法一都沒有符合音節數的詞語
            if len(match_word_1) == 0:
                synonyms = [] # 採用方法二
                match_word_2 = []
                if len(wordnet.synsets(word)) != 0: # 方法二有包含此詞語
                    syns = wordnet.synsets(word)[0].name()
                    for syn in wordnet.synsets(word): # pos可指定詞性，pos='a'
                        for l in syn.lemmas():
                            synonyms.append(l.name())
                    synonyms2 = list(set(synonyms)) # 用set()把重複的去除掉
                    print('synonyms2:',synonyms2) # 方法二抓取結果(未篩選音節數)
                    for w in range(len(synonyms2)):
                        if syllable_count(synonyms2[w]) == s: # 方法二列出符合音節數的同義詞
                            #print(synonyms2[w],syllable_count(synonyms2[w]))
                            match_word_2.append(synonyms2[w])
                    try:
                        print('m2 match list:',match_word_2)
                        lst_rewrite.append(match_word_2[0].replace('_',' ')) # 取代'_'有
                    except:
                        lst_rewrite.append('+') # 方法一和方法二都沒有符合的
                        pass

                else: # 方法二沒有包含此詞語(即方法一和方法二都沒有符合的)
                    print('NONE')
                    lst_rewrite.append('=')
            print('-'*70)
        # 非詞語('')
        else:
            lst_rewrite.append('')

    print(len(lst_rewrite))   
    print(lst_rewrite)
    return lst_rewrite

lst_rewrite = main_comparsion(tmp_wo)

original word: lived
correct syllable count: 1
crawl: ['alive', 'animate', 'breathing', 'living', 'aware', 'conscious', 'vital']
m1 match list: []
synonyms2: ['inhabit', 'survive', 'last', 'be', 'experience', 'exist', 'hold_out', 'go', 'know', 'live_on', 'live', 'dwell', 'populate', 'hold_up', 'endure', 'subsist']
m2 match list: ['last', 'be', 'go', 'know', 'live', 'dwell']
----------------------------------------------------------------------
original word: used
correct syllable count: 1
crawl: ['recycled', 'worn', 'hand-me-down', 'nearly new', 'not new', 'passed down', 'pre-owned']
m1 match list: ['worn']
----------------------------------------------------------------------
original word: woods
correct syllable count: 3
crawl: ['bosk', 'coppice', 'copse', 'silva', 'thicket', 'woodland', 'boscage']
m1 match list: []
synonyms2: ['forest', 'Sir_Henry_Joseph_Wood', 'Sir_Henry_Wood', 'woods', 'Mrs._Henry_Wood', 'woodwind', 'Grant_Wood', 'Natalie_Wood', 'woodwind_instrument', 'Wood', 'Ell

crawl: ['nay', 'nix', 'never', 'not']
m1 match list: []
synonyms2: ['no', 'atomic_number_102', 'no_more', 'No', 'nobelium']
m2 match list: ['nobelium']
----------------------------------------------------------------------
original word: a
correct syllable count: 2
crawl: []
1524 (total) / 6 (type) / 709 (s1) / 493 (s2) / 227 (s3) / 79 (s4) / 15 (s5) / 1 (s6)
s1 = ['people', 'way', 'art', 'world', 'map', 'two', 'health', 'meat', 'year', 'thanks', 'food', 'law', 'bird', 'love', 'science', 'fact', 'thing', 'player', 'week', 'movie', 'thought', 'child', 'month', 'truth', 'goal', 'news', 'growth', 'night', 'disk', 'road', 'role', 'soup', 'math', 'wood', 'flight', 'length', 'cell', 'lake', 'phone', 'scene', 'death', 'mood', 'blood', 'skill', 'wealth', 'depth', 'heart', 'ad', 'debt', 'loss', 'steak', 'seat', 'match', 'bread', 'employee', 'guest', 'height', 'mall', 'sample', 'mom', 'church', 'coffee', 'hair', 'lab', 'mode', 'mud', 'queen', 'song', 'tooth', 'town', 'wife', 'gate', 'girl', 'hal

crawl: ['spoken', 'aforementioned', 'aforesaid', 'forenamed']
m1 match list: []
synonyms2: ['suppose', 'order', 'enounce', 'sound_out', 'aver', 'said', 'pronounce', 'enunciate', 'aforementioned', 'aforesaid', 'read', 'allege', 'enjoin', 'tell', 'state', 'articulate', 'say']
m2 match list: ['enunciate']
----------------------------------------------------------------------
original word: my
correct syllable count: 2
crawl: ['y', 'ma', 'mi', 'mo', 'mc', 'md', 'mg', 'mn']
m1 match list: []
NONE
----------------------------------------------------------------------
original word: And
correct syllable count: 2
crawl: ['along with', 'also', 'as a consequence', 'as well as', 'furthermore', 'including', 'moreover', 'together with']
m1 match list: ['also']
----------------------------------------------------------------------
original word: find
correct syllable count: 2
crawl: ['treasure trove', 'acquisition', 'asset', 'bargain', 'boast', 'bonanza', 'catch', 'gem', 'jewel', 'pride', 'treasure'

crawl: ['directed toward', 'facing', 'into', 'through', 'toward', 'via']
m1 match list: ['facing', 'into', 'toward']
----------------------------------------------------------------------
original word: But
correct syllable count: 2
crawl: ['although', 'however', 'nevertheless', 'on the other hand', 'still', 'though', 'yet']
m1 match list: ['although']
----------------------------------------------------------------------
original word: somehow
correct syllable count: 1
crawl: ['anyway', 'after a fashion', 'anyhow', 'anywise', 'by hook or crook', 'come what may', 'in one way or another', 'in some such way', 'in some way', 'one way or another', 'somehow or another', 'somehow or other']
m1 match list: []
synonyms2: ['someways', 'in_some_way', 'somehow', 'in_some_manner', 'someway']
m2 match list: []
----------------------------------------------------------------------
original word: what
correct syllable count: 2
crawl: ['hat', 'whet', 'whit', 'chat', 'shat', 'that', 'wham', 'what i']
m

crawl: ['directed toward', 'facing', 'into', 'through', 'toward', 'via']
m1 match list: ['facing', 'into', 'toward']
----------------------------------------------------------------------
original word: down
correct syllable count: 2
crawl: ['downward', 'cascading', 'declining', 'depressed', 'descending', 'downgrade', 'downhill', 'dropping', 'falling', 'gravitating', 'inferior', 'precipitating', 'sagging', 'sinking', 'sliding', 'slipping', 'bottomward', 'earthward', 'groundward', 'nether', 'subjacent', 'to the bottom', 'under', 'underneath']
m1 match list: ['downward', 'downgrade', 'downhill', 'dropping', 'falling', 'sagging', 'sinking', 'sliding', 'slipping', 'earthward', 'groundward', 'nether', 'under']
----------------------------------------------------------------------
original word: don't
correct syllable count: 2
crawl: ['constraint', 'embargo', 'exclusion', 'injunction', 'prevention', 'proscription', 'restriction', 'taboo', 'veto', 'bar', 'interdict', 'interdiction', 'negation

crawl: ['constraint', 'embargo', 'exclusion', 'injunction', 'prevention', 'proscription', 'restriction', 'taboo', 'veto', 'bar', 'interdict', 'interdiction', 'negation', 'obstruction', 'refusal', 'repudiation', 'temperance', 'disallowance', 'no-no', 'off limits', 'out of bounds']
m1 match list: ['constraint', 'taboo', 'veto', 'no-no']
----------------------------------------------------------------------
original word: I
correct syllable count: 2
crawl: ['amplify', 'boast', 'boost', 'brag', 'caricature', 'color', 'corrupt', 'distort', 'embroider', 'emphasize', 'enlarge', 'exalt', 'expand', 'fabricate', 'falsify', 'fudge', 'heighten', 'hike', 'inflate', 'blow out of proportion', 'blow up out of proportion', 'build up', 'cook up', 'go to extremes']
m1 match list: ['color', 'corrupt', 'distort', 'enlarge', 'exalt', 'expand', 'heighten', 'inflate', 'build up', 'cook up']
----------------------------------------------------------------------
original word: the
correct syllable count: 3
craw

In [43]:
# 產生重製的歌詞
def reproduce_lyrics(tmp_w,lst_rewrite):
    final = []
    for p in range(len(tmp_w)):
        if tmp_w[p] == '':
            final.append(lst_rewrite[p])
        else:
            final.append(tmp_w[p])
    print(len(final)) # 型態為list
    #print(final)
    output = ' '.join(final) # list轉換成string
    print(output)
    return final,output

final,output = reproduce_lyrics(tmp_w,lst_rewrite)

277
Where the old men pray His children last next door They worn to live in the Mrs. Henry Wood But this manes pious bone He didn't have public prayer nobelium he didn't have reward prayer He just enunciate I've got to find = own way home also I've got to asset = own way home If I can do comic things he says That I just can't get Then I'll just settle down and kickoff brisk because today Lord how I'd love to settle down And start brisk because now Lord how I'd love to settle down But I don't have a prayer I don't have a prayer never color don't have a prayer + color think of what a partner must be And = I hate = do things That he docent have facing do although then + I think of what i hope is And do I have to start Oh oh I'm apt to create Oh = I'd love to settle downward I'm apt apt apt If I bucket do the things he says That color just can't get again I'll just settle down and start brisk for today Lord how I'd love facing settle downward And I don't have a prayer No I constraint have 

**<font color = blue size=5>修改符號地方</font>**

In [44]:
# 修改output有符號的地方
def fix_symbol(final):
    lst_fix = []
    lst_fix_symbol = []
    for k in range(len(final)):
        fix = []
        if final[k] == '+' or final[k] == '=': # 當遇到'+'或'='符號
            fix = ' '.join(final[k-4:k]) # 沒包含'+'和'='符號 (k)
            fix_symbol = ' '.join(final[k-4:k+1]) # 有包含'+'和'='符號 (k+1)
            lst_fix.append(fix)
            lst_fix_symbol.append(fix_symbol)
    print(len(lst_fix))
    print('lst_fix:',lst_fix)
    print(len(lst_fix_symbol))
    print('lst_fix_symbol:',lst_fix_symbol)
    return lst_fix,lst_fix_symbol

lst_fix,lst_fix_symbol = fix_symbol(final)

8
lst_fix: ["I've got to find", "I've got to asset", "don't have a prayer", 'partner must be And', 'And = I hate', 'facing do although then', 'apt to create Oh', 'constraint have a prayer']
8
lst_fix_symbol: ["I've got to find =", "I've got to asset =", "don't have a prayer +", 'partner must be And =', 'And = I hate =', 'facing do although then +', 'apt to create Oh =', 'constraint have a prayer +']


In [47]:
# 需要修改的字詞片段儲存到output/fix的txt檔案
# https://stackabuse.com/reading-and-writing-lists-to-a-file-in-python/
def save_file(filename,lst_fix):
    with open(filename, 'w') as filehandle:
        for listitem in lst_fix:
            filehandle.write('%s\n' % listitem)
    print('Save to ' + filename)

filename = 'output/fix/fixpart01.txt'
save_file(filename,lst_fix)

Save to output/fix/fixpart01.txt


轉換到 predict_new_word.ipynb 處理

In [48]:
# 讀取output/fix的txt檔案(經過predict_new_word.ipynb處理後的檔案)
def read_file(filename):
    places = []
    with open(filename, 'r') as filehandle:
        for line in filehandle:
            currentPlace = line[:-1] # 移除換行符號
            places.append(currentPlace)
    print('Read from ' + filename)
    print(len(places))
    print('places:',places)
    return places

filename = 'output/fix/fixpart01-1.txt'
places = read_file(filename)

Read from output/fix/fixpart01-1.txt
8
places: ["I've got to find to", "I've got to asset to", "don't have a prayer have", 'partner must be And and', 'And start I hate to', 'facing do although then the', 'apt to create Oh settle', 'constraint have a prayer a']


In [55]:
# 利用lst_fix_symbol到output歌詞去修改文本output2
def output_final(output,lst_fix_symbol,places):
    output2 = output
    for r in range(len(lst_fix_symbol)):
        print('Ori:',lst_fix_symbol[r],'/ Fix:',places[r])
        output2 = output2.replace(lst_fix_symbol[r],places[r])
    print('\n' + output2)
    return output2

output2 = output_final(output,lst_fix_symbol,places)

Ori: I've got to find = / Fix: I've got to find to
Ori: I've got to asset = / Fix: I've got to asset to
Ori: don't have a prayer + / Fix: don't have a prayer have
Ori: partner must be And = / Fix: partner must be And and
Ori: And = I hate = / Fix: And start I hate to
Ori: facing do although then + / Fix: facing do although then the
Ori: apt to create Oh = / Fix: apt to create Oh settle
Ori: constraint have a prayer + / Fix: constraint have a prayer a

Where the old men pray His children last next door They worn to live in the Mrs. Henry Wood But this manes pious bone He didn't have public prayer nobelium he didn't have reward prayer He just enunciate I've got to find to own way home also I've got to asset to own way home If I can do comic things he says That I just can't get Then I'll just settle down and kickoff brisk because today Lord how I'd love to settle down And start brisk because now Lord how I'd love to settle down But I don't have a prayer I don't have a prayer never color d

# -

In [8]:
# 生成歌詞
gen_text = "Where the old men pray\
            His children lived next door\
            They used to live in the woods\
            But this man's religious bone\
            He didn't have a prayer\
            No, he didn't have a prayer\
            He just said I've got to find my own way home\
            And I've got to find my own way home\
            If I can do the things he says\
            That I just can't understand\
            Then I'll just settle down and start living for today\
            Lord, how I'd love to settle down\
            And start living for today\
            Lord, how I'd love to settle down\
            But I don't have a prayer\
            I don't have a prayer\
            No, I don't have a prayer\
            Sometimes, I think of what a wife must be\
            And how I hate to do things\
            That he doesn't have to do\
            But then somehow, I think of what hope is\
            And do I have to start\
            Oh, oh, I'm ready to begin\
            Oh, how I'd love to settle down\
            I'm ready, ready, ready\
            If I can do the things he says\
            That I just can't understand\
            Then I'll just settle down and start living for today\
            Lord, how I'd love to settle down\
            And I don't have a prayer\
            No, I don't have a prayer\
            Always, always start the day\
            Lord, how I'd love to settle down\
            I'm ready, ready, ready\
            If I can do the things he says\
            That I just can't understand\
            Then I'll just settle down and start living for today\
            Lord, how I'd love to settle down\
            But I don't have a prayer No, I don't have a prayer\
            I can't do the things he says\
            That I just can't understand\
            Then I'll just settle down and start living for today"
tokenizer = RegexpTokenizer("[\w']+") 
lst_gen_tokenize = tokenizer.tokenize(gen_text)
print(len(lst_gen_tokenize))
print(lst_gen_tokenize)

288
['Where', 'the', 'old', 'men', 'pray', 'His', 'children', 'lived', 'next', 'door', 'They', 'used', 'to', 'live', 'in', 'the', 'woods', 'But', 'this', "man's", 'religious', 'bone', 'He', "didn't", 'have', 'a', 'prayer', 'No', 'he', "didn't", 'have', 'a', 'prayer', 'He', 'just', 'said', "I've", 'got', 'to', 'find', 'my', 'own', 'way', 'home', 'And', "I've", 'got', 'to', 'find', 'my', 'own', 'way', 'home', 'If', 'I', 'can', 'do', 'the', 'things', 'he', 'says', 'That', 'I', 'just', "can't", 'understand', 'Then', "I'll", 'just', 'settle', 'down', 'and', 'start', 'living', 'for', 'today', 'Lord', 'how', "I'd", 'love', 'to', 'settle', 'down', 'And', 'start', 'living', 'for', 'today', 'Lord', 'how', "I'd", 'love', 'to', 'settle', 'down', 'But', 'I', "don't", 'have', 'a', 'prayer', 'I', "don't", 'have', 'a', 'prayer', 'No', 'I', "don't", 'have', 'a', 'prayer', 'Sometimes', 'I', 'think', 'of', 'what', 'a', 'wife', 'must', 'be', 'And', 'how', 'I', 'hate', 'to', 'do', 'things', 'That', 'he', "

In [9]:
# 計算生成歌詞的音節數量
lst_gen_syllables = []
count1 = []
count2 = []
count3 = []
for i in range(len(lst_gen_tokenize)):
    count_gen = syllable_count(lst_gen_tokenize[i])
    lst_gen_syllables.append(count_gen)
print('lst_gen_syllables:',len(lst_gen_syllables))
print(lst_gen_syllables)

lst_gen_syllables: 288
[1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2]


In [11]:
# 原始歌詞
ori_text = "I'll let you show me his moves\
            Let you do what he taught you\
            Let you reminisce how you used, to do\
            Girl I?m open to anything that'll get you into that zone\
            And understand that we?re all alone\
            So you can slowly take off your clothes\
            Baby girl you know what's in store\
            And baby I will stay up all night\
            I've been goin' hard since last night\
            And I'mma go harder tonight\
            Wish you could see you through my eyes\
            Ooh I'm telling you this ain't the same\
            And I know he's still in your brain\
            I'm 'bout to burn that shit into flames\
            Once I'm in you\
            Baby\
            Forget what you know\
            Make yourself at home\
            'Cuz baby when I'm finished with ya you won't wanna go Outside\
            And I'mma work you like a pro, baby\
            And, you gon' take it like one\
            Yeah you gon' take it like one\
            And I'mma give it like you asked for it\
            Why, cuz you been talkin' 'bout it\
            I know you been talkin' 'bout it\
            Ooh baby when I'm done with you\
            You ain't saying nothing\
            Yeah you ain't saying nothing\
            Ooh baby when I'm done with you\
            You ain't saying nothing\
            Yeah you ain't saying nothing\
            You gon' make me show off\
            All the pain that you feel you can tell that we ain't making no love\
            But I'll pretend\
            Oh girl, I'll pretend\
            If you pretend then girl I'll pretend\
            Let's make it seem like we're all we need in the end\
            Forget what you know\
            Make yourself at home\
            'Cuz baby when I'm finished with ya you won't wanna go Outside"
tokenizer = RegexpTokenizer("[\w']+") 
lst_ori_tokenize = tokenizer.tokenize(ori_text)
print(len(lst_ori_tokenize))
print(lst_ori_tokenize)

277
["I'll", 'let', 'you', 'show', 'me', 'his', 'moves', 'Let', 'you', 'do', 'what', 'he', 'taught', 'you', 'Let', 'you', 'reminisce', 'how', 'you', 'used', 'to', 'do', 'Girl', 'I', 'm', 'open', 'to', 'anything', "that'll", 'get', 'you', 'into', 'that', 'zone', 'And', 'understand', 'that', 'we', 're', 'all', 'alone', 'So', 'you', 'can', 'slowly', 'take', 'off', 'your', 'clothes', 'Baby', 'girl', 'you', 'know', "what's", 'in', 'store', 'And', 'baby', 'I', 'will', 'stay', 'up', 'all', 'night', "I've", 'been', "goin'", 'hard', 'since', 'last', 'night', 'And', "I'mma", 'go', 'harder', 'tonight', 'Wish', 'you', 'could', 'see', 'you', 'through', 'my', 'eyes', 'Ooh', "I'm", 'telling', 'you', 'this', "ain't", 'the', 'same', 'And', 'I', 'know', "he's", 'still', 'in', 'your', 'brain', "I'm", "'bout", 'to', 'burn', 'that', 'shit', 'into', 'flames', 'Once', "I'm", 'in', 'you', 'Baby', 'Forget', 'what', 'you', 'know', 'Make', 'yourself', 'at', 'home', "'Cuz", 'baby', 'when', "I'm", 'finished', 'wit

In [12]:
# 計算原始歌詞的音節數量
lst_ori_syllables = []
for i in range(len(lst_ori_tokenize)):
    count_ori = syllable_count(lst_ori_tokenize[i])
    lst_ori_syllables.append(count_ori)
print(len(lst_ori_syllables))
print(lst_ori_syllables)

277
[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 2, 1, 2]


In [19]:
data = """ Where the old men pray\n
        His children lived next door\n
        They used to live in the woods\n
        But this man's religious bone\n
        He didn't have a prayer\n
        No, he didn't have a prayer\n
        He just said I've got to find my own way home\n
        And I've got to find my own way home\n
        If I can do the things he says\n
        That I just can't understand\n
        Then I'll just settle down and start living for today\n
        Lord, how I'd love to settle down\n
        And start living for today\n
        Lord, how I'd love to settle down\n
        But I don't have a prayer\n
        I don't have a prayer\n
        No, I don't have a prayer\n
        Sometimes, I think of what a wife must be\n
        And how I hate to do things\n
        That he doesn't have to do\n
        But then somehow, I think of what hope is\n
        And do I have to start\n
        Oh, oh, I'm ready to begin\n
        Oh, how I'd love to settle down\n
        I'm ready, ready, ready\n
        If I can do the things he says\n
        That I just can't understand\n
        Then I'll just settle down and start living for today\n
        Lord, how I'd love to settle down\n
        And I don't have a prayer\n
        No, I don't have a prayer\n
        Always, always start the day\n
        Lord, how I'd love to settle down\n
        I'm ready, ready, ready\n
        If I can do the things he says\n
        That I just can't understand\n
        Then I'll just settle down and start living for today\n
        Lord, how I'd love to settle down\n
        But I don't have a prayer No, I don't have a prayer\n
        I can't do the things he says\n
        That I just can't understand\n
        Then I'll just settle down and start living for today\n """
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [20]:
seed_text = "I've got to find"
print(generate_seq(model, tokenizer, seed_text, 1))

InternalError: Blas GEMM launch failed : a.shape=(4, 50), b.shape=(50, 50), m=4, n=50, k=50
	 [[Node: lstm_2/while/MatMul_7 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](lstm_2/while/Switch_3:1, lstm_2/while/MatMul_7/Enter)]]
	 [[Node: dense_2/Softmax/_205 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_259_dense_2/Softmax", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [1]:
for v in range(0,1):
    seed_text = lst_fix[v]
    fix_result = generate_seq(model, tokenizer, seed_text, 1)
    print(fix_result)

NameError: name 'lst_fix' is not defined

In [48]:
# 搜尋詞語的同義詞(使用NLTK)
#nltk.download('wordnet')
num = 40
synonyms = []
match_word = []
word = tmp_wo[num]
try:
    if len(wordnet.synsets(word)) != 0:
        syns = wordnet.synsets(word)[0].name()
    else:
        crawl = crawl_synonyms(tmp_wo[num])
        s = tmp_so[num]
        for n in range(len(crawl)):
            if syllable_count(crawl[n]) == s: # 列出符合音節數的同義詞
                print(crawl[n],syllable_count(crawl[n]))
                match_word.append(crawl[n])
        if len(match_word) == 0:
            match_word.append('-')
except:
    word = preprocessText2(word) # 對詞語預處理
    syns = wordnet.synsets(word)[0].name()
print('original word:',word)
for syn in wordnet.synsets(word): # pos可指定詞性,pos='a'
    for l in syn.lemmas():
        synonyms.append(l.name())
synonyms2 = list(set(synonyms))
print('synonyms list:',synonyms2) # 用set()把重複的去除掉

s = tmp_so[num] # 正確的音節數量
print('correct syllable count:',s)

for n in range(len(synonyms2)):
    if syllable_count(synonyms2[n]) == s: # 列出符合音節數的同義詞
        print(synonyms2[n],syllable_count(synonyms2[n]))
        match_word.append(synonyms2[n])
if len(match_word) == 0:
    crawl = crawl_synonyms(tmp_wo[num])
    for n in range(len(crawl)):
        if syllable_count(crawl[n]) == s: # 列出符合音節數的同義詞
            print(crawl[n],syllable_count(crawl[n]))
            match_word.append(crawl[n])
print('match_word:',match_word)

original word: my
synonyms list: []
correct syllable count: 2
match_word: ['-']


In [55]:
# (舊版本)搜尋詞語的同義詞(使用NLTK)，並更新list(lst_rewrite)
lst_rewrite = []
match_word = []

for num in range(len(tmp_wo)): # len(tmp_wo)
    synonyms = []
    word = tmp_wo[num]
    if word != '':
        try:
            if len(wordnet.synsets(word)) != 0:
                syns = wordnet.synsets(word)[0].name()
            else:
                crawl = crawl_synonyms(tmp_wo[num])
                print('wordnet.synsets NONE')
                s = tmp_so[num]
                for n in range(len(crawl)):
                    if syllable_count(crawl[n]) == s: # 列出符合音節數的同義詞
                        #print(crawl[n],syllable_count(crawl[n]))
                        match_word.append(crawl[n])
                if len(match_word) == 0:
                    lst_rewrite.append('-')
        except:
            word = preprocessText2(word) # 對詞語預處理
            syns = wordnet.synsets(word)[0].name()
        print('original word:',word)
        for syn in wordnet.synsets(word): # pos可指定詞性,pos='a'
            for l in syn.lemmas():
                synonyms.append(l.name())
        synonyms2 = list(set(synonyms))
        print('synonyms list:',synonyms2) # 用set()把重複的去除掉

        s = tmp_so[num] # 正確的音節數量
        print('correct syllable count:',s)
        match_word = []
        for n in range(len(synonyms2)):
            if syllable_count(synonyms2[n]) == s: # 列出符合音節數的同義詞
                #print(synonyms2[n],syllable_count(synonyms2[n]))
                match_word.append(synonyms2[n])
        
        if len(match_word) != 0: # 用wordnet方法找的到同義詞(match_word陣列長度不為0)
            print('match word:',match_word[0])
            lst_rewrite.append(match_word[0])
        else:  # 方法一(wordnet方法)找不到同義詞，採用第二種方法抓取同義詞(def crawl_synonyms)
            crawl = crawl_synonyms(tmp_wo[num])
            for n in range(len(crawl)):
                if syllable_count(crawl[n]) == s: # 列出符合音節數的同義詞
                    match_word.append(crawl[n])
            try:
                print('match word:',match_word[0])
                lst_rewrite.append(match_word[0])
            except:
                pass
        print('-'*70)
    else:
        lst_rewrite.append('')

print(len(lst_rewrite))   
print(lst_rewrite)

original word: lived
synonyms list: ['dwell', 'hold_up', 'go', 'subsist', 'survive', 'live_on', 'populate', 'exist', 'experience', 'endure', 'live', 'know', 'last', 'hold_out', 'be', 'inhabit']
correct syllable count: 1
match word: dwell
----------------------------------------------------------------------
original word: used
synonyms list: ['apply', 'use', 'habituate', 'secondhand', 'ill-used', 'used', 'victimised', 'utilise', 'utilize', 'exploited', 'practice', 'employ', 'put-upon', 'expend', 'victimized']
correct syllable count: 1
match word: use
----------------------------------------------------------------------
original word: woods
synonyms list: ['Sir_Henry_Wood', 'Wood', 'Mrs._Henry_Wood', 'woodwind', 'woods', 'Sir_Henry_Joseph_Wood', 'Natalie_Wood', 'Grant_Wood', 'Ellen_Price_Wood', 'woodwind_instrument', 'wood', 'forest']
correct syllable count: 3
match word: Mrs._Henry_Wood
----------------------------------------------------------------------
wordnet.synsets NONE
origina

----------------------------------------------------------------------
original word: down
synonyms list: ['down', 'dispirited', 'low-spirited', 'shoot_down', 'depressed', 'downwardly', 'grim', 'mastered', 'down_pat', 'pour_down', 'toss_off', 'bolt_down', 'John_L._H._Down', 'down_in_the_mouth', 'kill', 'consume', 'downwards', 'belt_down', 'downcast', 'knock_down', 'down_feather', 'push_down', 'pile', 'pull_down', 'devour', 'downward', 'gloomy', 'fine-tune', 'low', 'go_through', 'drink_down', 'Down', 'blue', 'pop', 'land', 'downhearted', 'cut_down', 'polish', 'refine']
correct syllable count: 2
match word: shoot_down
----------------------------------------------------------------------
original word: ready
synonyms list: ['prepare', 'ready', 'quick', 'set', 'set_up', 'cook', 'fix', 'make', 'gear_up']
correct syllable count: 1
match word: quick
----------------------------------------------------------------------
original word: ready
synonyms list: ['prepare', 'ready', 'quick', 'set', 

In [16]:
# 透過predict_new_word.ipynb訓練的詞語模型(替換data去訓練)
model = load_model('model/model_predictwords.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1, 10)             780       
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_2 (Dense)              (None, 78)                3978      
Total params: 16,958
Trainable params: 16,958
Non-trainable params: 0
_________________________________________________________________


In [17]:
# 預測字詞函式
def generate_seq(model, tokenizer, seed_text, n_words):
    in_text, result = seed_text, seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = np.array(encoded)
        # predict a word in the vocabulary
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if any(index == yhat):
                out_word = word
                break
        # append to input
        in_text, result = out_word, result + ' ' + out_word
    return result

In [100]:
# 產生表格比較
from pandas import ExcelWriter
with ExcelWriter('df/df.xlsx') as writer:
    df1 = pd.DataFrame(lst_rewrite)
    df2 = pd.DataFrame(tmp_wo)
    df3 = pd.DataFrame(lst_wrong)
    df4 = pd.DataFrame({'lst_rewrite': lst_rewrite,'tmp_wo': tmp_wo,'lst_wrong': lst_wrong})
    df1.to_excel(writer, 'lst_rewrite')
    df2.to_excel(writer, 'tmp_wo')
    df3.to_excel(writer, 'lst_wrong')
    df4.to_excel(writer, 'all')
    writer.save()